In [14]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.optimizers.schedules import ExponentialDecay
# from tensorflow.keras.losses import BinaryCrossentropyLoss
from tqdm import trange
import matplotlib.pyplot as plt


class FullyConnectedNet(Model):
    def __init__(self, hidden_dim, output_dim, activation='sigmoid'):
        super(FullyConnectedNet, self).__init__()
        self.dense1 = Dense(hidden_dim, activation=activation)
        self.out = Dense(output_dim, activation='sigmoid')
    def call(self,x):
        x = self.dense1(x)
        x = self.out(x)
        return x


def train(model, dataset, loss_obj, num_epochs, optimizer, train_loss, train_accuracy,
    test_dataset, test_loss, test_accuracy):
    
    arr_train_loss = np.zeros((num_epochs))
    arr_train_acc = np.zeros((num_epochs))
    arr_test_loss = np.zeros((num_epochs))
    arr_test_acc = np.zeros((num_epochs))
    for epoch in trange(num_epochs):
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()
        for X_data, labels in dataset.shuffle(10000).batch(50):
            with tf.GradientTape() as tape:
                pred = model(X_data, training=True)
                loss = loss_obj(labels, pred)
            
            grad = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grad, model.trainable_variables))

            train_loss(labels, pred)
            train_accuracy(labels, tf.squeeze(pred, axis=1))
        
        arr_train_acc[epoch] = train_accuracy.result() * 100
        arr_train_loss[epoch] = train_loss.result()

        for X_data, labels in test_dataset.shuffle(121345).batch(50):
            pred = model(X_data, training=False)
            loss = loss_obj(labels, pred)

            test_loss(labels, pred)
            test_accuracy(labels, tf.squeeze(pred, axis=1))
        arr_test_acc[epoch] = test_accuracy.result() * 100
        arr_test_loss[epoch] = test_loss.result()
        
        # if epoch%1 == 0:
        #     print(
        #         f'Epoch {epoch + 1}, '
        #         f'Loss: {train_loss.result()}, '
        #         f'Accuracy: {train_accuracy.result() * 100}'
        #     )
    plt.figure()
    plt.plot(np.arange(num_epochs),arr_train_acc, '-o', label="Train")
    plt.plot(np.arange(num_epochs), arr_test_acc, '-x', label="Test")
    plt.set_title("Accuracy")
    plt.show()

    plt.figure()
    plt.plot(np.arange(num_epochs),arr_train_loss, '-o', label="Train")
    plt.plot(np.arange(num_epochs), arr_test_loss, '-x', label="Test")
    plt.set_title("Loss")
    plt.show()
# def test(model, dataset, loss_obj, num_epochs, optimizer, train_loss, train_accuracy):


lr = ExponentialDecay(2.0, decay_steps=10**2, decay_rate=0.9, name="exponential_decay_schedule")
num_epochs = 50
dtype = tf.float64
model = FullyConnectedNet(2,1,'sigmoid')

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Nadam(learning_rate=lr)
train_loss = tf.keras.metrics.BinaryCrossentropy()
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='binary_accuracy')
test_loss = tf.keras.metrics.BinaryCrossentropy()
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='binary_accuracy')
training_data = np.loadtxt("training_data.txt")
test_data = np.loadtxt("test_data.txt")
training_data = tf.convert_to_tensor(training_data, dtype=dtype)
test_data = tf.convert_to_tensor(test_data, dtype=dtype)
X_train, y_train = training_data[:,1:], training_data[:,0]
X_test, y_test = training_data[:,1:], training_data[:,0]

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)
)


test_ds = tf.data.Dataset.from_tensor_slices(
    (X_test, y_test)
)

train(model, train_ds, loss_object, num_epochs, optimizer, train_loss, train_accuracy, 
    test_ds, test_loss, test_accuracy)







 18%|█▊        | 9/50 [00:21<01:39,  2.42s/it]